<a href="https://colab.research.google.com/github/alks444/A3-ESILV/blob/main/Puissance4_ANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
from math import inf
from colorama import *
import time

In [17]:
def grille(hauteur,largeur):
    Grille=[]
    for i in range(hauteur):
        l=[]
        for j in range(largeur):
            l+=[0]
        Grille+=[l]
    return Grille


def action(grille,indice,player):
    L=[]
    for elem in grille:
        L+=[elem]
    k=-1
    while L[k][indice]!=0 and k>-len(grille):
        k=k-1
    if player != "annuler":
        L[k][indice]=player
        return L
    if player == "annuler":
        if indice in listeaction(grille):
            L[k+1][indice]=0
            return L
        if indice not in listeaction(grille):
            L[k][indice]=0
            return L


def affichemat(gr):
    c=""
    for i in range(len(gr[0])):
        c+="---"
    print(c)
    for elem in gr:
        print(elem)
    print(c)


def affichegr(gr):
    c=" "
    for i in range(len(gr[0])):
        c+=f"-{i}-"
    print(c)
    for elem in gr:
        st=""
        for el in elem:
            val = f"|{Back.RED+Fore.RED}X{Style.RESET_ALL}" if el ==1 else "| " if el ==0 else f"|{Back.CYAN+Fore.CYAN}O{Style.RESET_ALL}"
            st+="|"+str(val)
        print(st+"||")


def listeaction(gr):
    return [i for i in range(len(gr[0])) if gr[0][i]==0]


def listeactionjoueur(laction):
    return [elem+1 for elem in laction]


def findepartie(grille):
    if len(listeaction(grille))==0:
        return True
    if len(listeaction(grille))!=0:
        return False


def remplircol(grille,col,player):
    for i in range(len(grille)):
        grille=action(grille,col-1,player)
    return grille

In [18]:
def utility(grille):
    largeurvide=[]
    evalu=0
    for hauteur in range(len(grille)):
        h = len(grille) - hauteur - 1
        for l in range(len(grille[0])):
            if l not in largeurvide:
                if grille[h][l]!=0:

                    ### CONDITION DE VICTOIRE
                    if l<len(grille[0])-3:
                        if grille[h][l]==grille[h][l+1]==grille[h][l+2]==grille[h][l+3]:
                            return grille[h][l]*1000000,True
                    if h > len(grille)-4:
                        if grille[h][l]==grille[h-1][l]==grille[h-2][l]==grille[h-3][l]:
                            return grille[h][l]*1000000,True
                        if l>2:
                            if grille[h][l]==grille[h-1][l-1]==grille[h-2][l-2]==grille[h-3][l-3]:
                                return grille[h][l]*1000000,True
                        if l<len(grille[0])-3:
                            if grille[h][l]==grille[h-1][l+1]==grille[h-2][l+2]==grille[h-3][l+3]:
                                return grille[h][l]*1000000,True

                    ## Grille Heuristique
                    evalu+=grille[h][l]*Heuri[h][l]

                if grille[h][l]==0:
                    largeurvide+=[l]
    return evalu,False


def matriceheuristique(grille):
    heuri=[]
    for hauteur in range(len(grille)):
        ligne=[]
        h = len(grille) - hauteur - 1
        for l in range(len(grille[0])):
            val=0

            if h > len(grille)-4 and h< len(grille):
                val+=1
            if h > len(grille)-5 and h< len(grille)-1:
                val+=1
            if h > len(grille)-6 and h< len(grille)-2:
                val+=1
            if h > len(grille)-7 and h< len(grille)-3:
                val+=1

            if l<len(grille[0])-3 and l>=3:
                val+=1
            if l<len(grille[0])-2 and l>=2:
                val+=1
            if l<len(grille[0])-1 and l>=1:
                val+=1
            if l<len(grille[0]) and l>=0:
                val+=1

            if (h > len(grille)-4 and h< len(grille)) and (l<len(grille[0])-3 and l>=3):
                val+=1
            if (h > len(grille)-5 and h< len(grille)-1) and (l<len(grille[0])-2 and l>=2):
                val+=1
            if (h > len(grille)-6 and h< len(grille)-2) and (l<len(grille[0])-1 and l>=1):
                val+=1
            if (h > len(grille)-7 and h< len(grille)-3) and (l<len(grille[0]) and l>=0):
                val+=1

            if (h > len(grille)-4 and h< len(grille)) and (l<len(grille[0]) and l>=0):
                val+=1
            if (h > len(grille)-5 and h< len(grille)-1) and (l<len(grille[0])-1 and l>=1):
                val+=1
            if (h > len(grille)-6 and h< len(grille)-2) and (l<len(grille[0])-2 and l>=2):
                val+=1
            if (h > len(grille)-7 and h< len(grille)-3) and (l<len(grille[0])-3 and l>=3):
                val+=1

            ligne+=[val]
        heuri+=[ligne]
    return heuri

In [19]:
def ajouter(grille):
    a=0
    affichegr(grille)
    print("colonnes")
    a=int(input())
    print("player")
    b=int(input())
    while a!="s":
        grille=action(grille,int(a)-1,b)
        affichegr(grille)
        print("colonnes")
        a=input()
        print("player")
        b=int(input())


In [20]:
def minmax(grille,depth,maximizing,alpha,beta):

    if maximizing == True:
        player= 1
        val = -float("inf")

    if maximizing == False:
        player = -1
        val = float("inf")

    if utility(grille)[1]==True:
        return utility(grille)[0]*(depth+2),None

    if depth == 0 or findepartie(grille)==True:
        return utility(grille)[0],None

    if maximizing == True:
        for noeud in listeaction(grille):
            grille=action(grille,noeud,player)
            valeur_eph= minmax(grille,depth-1,False,alpha,beta)[0]
            grille=action(grille,noeud,"annuler")

            if val < valeur_eph:
                val=valeur_eph
                meilleurcoup=noeud
            alpha = max(val,alpha)
            if alpha >= beta:
                break
            if 'meilleurcoup' not in locals():
                meilleurcoup = listeaction(grille)[0]

    if maximizing == False:
        for noeud in listeaction(grille):
            grille=action(grille,noeud,player)
            valeur_eph= minmax(grille,depth-1,True,alpha,beta)[0]
            grille=action(grille,noeud,"annuler")

            if val > valeur_eph:
                val=valeur_eph
                meilleurcoup=noeud
            beta = min(val,beta)
            if alpha>=beta:
                break
            if 'meilleurcoup' not in locals():
                meilleurcoup = listeaction(grille)[0]
    return val,meilleurcoup

In [38]:
def jouervsordi(haut=6,larg=12,depth=3):
    temps_jeu = 0
    gri=grille(haut,larg)
    global Heuri
    Heuri=matriceheuristique(gri)
    compteur=0
    print("Jouer en premier? (Y/N)")
    a=input()
    if a=='y' or a=="Y":
        player = 1
        ordimax=False
        ordi=-1
    elif a=="N" or a=="n":
        player = -1
        ordimax=True
        ordi=1
        start = time.time()
        coupod=minmax(gri,depth,ordimax,-float("inf"),float("inf"))[1]
        end = time.time()
        temps_jeu+=(end-start)
        print(f"\nl'ordi joue {coupod}\net a mit {end-start} secondes à jouer")
        print(f"temps de jeu total : {temps_jeu}\n")
        gri=action(gri,coupod,ordi)
        compteur += 1
    else:
        return"stop partie role"


    while findepartie(gri)==False and utility(gri)[1]!=True:
        affichegr(gri)
        print(f"colonnes dipso {listeaction(gri)}")
        uwu=0
        while True:
            try:
                coupj=int(input())
                break
            except:
                uwu+=1
                if uwu==3:
                    return "STOP"
                print("Ce n'est pas une entrée valide")

        if coupj in listeaction(gri):
            gri=action(gri,coupj,player)
            compteur += 1
            print(f"\nle joueur joue {coupj}\n")
            affichegr(gri)
            if findepartie(gri)==False and utility(gri)[1]!=True:
                start = time.time()
                coupo=minmax(gri,depth,ordimax,-float("inf"),float("inf"))[1]
                end = time.time()
                gri=action(gri,coupo,ordi)
                compteur += 1
                temps_jeu+=(end-start)
                print(f"\nl'ordi joue {coupo}\net a mit {end-start} secondes à jouer")
                print(f"temps de jeu total : {temps_jeu}\n")
    affichegr(gri)
    if utility(gri)[1]==True:
        if utility(gri)[0]>0:
            return "Gagnant 1"
        if utility(gri)[0]<=0:
            return"Gagnant 2"

In [39]:
jouervsordi(6,12,5)

Jouer en premier? (Y/N)
y
 -0--1--2--3--4--5--6--7--8--9--10--11-
|| || || || || || || || || || || || ||
|| || || || || || || || || || || || ||
|| || || || || || || || || || || || ||
|| || || || || || || || || || || || ||
|| || || || || || || || || || || || ||
|| || || || || || || || || || || || ||
colonnes dipso [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
5

le joueur joue 5

 -0--1--2--3--4--5--6--7--8--9--10--11-
|| || || || || || || || || || || || ||
|| || || || || || || || || || || || ||
|| || || || || || || || || || || || ||
|| || || || || || || || || || || || ||
|| || || || || || || || || || || || ||
|| || || || || ||X|| || || || || || ||

l'ordi joue 3
et a mit 1.252120018005371 secondes à jouer
temps de jeu total : 1.252120018005371

 -0--1--2--3--4--5--6--7--8--9--10--11-
|| || || || || || || || || || || || ||
|| || || || || || || || || || || || ||
|| || || || || || || || || || || || ||
|| || || || || || || || || || || || ||
|| || || || || || || || || || || || ||
|| || || ||O|| ||

'Gagnant 2'